In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!git lfs install
!git clone https://huggingface.co/vinai/phobert-base

Error: Failed to call git rev-parse --git-dir --show-toplevel: "fatal: not a git repository (or any of the parent directories): .git\n"
Git LFS initialized.
Cloning into 'phobert-base'...
remote: Enumerating objects: 51, done.
remote: Counting objects: 100% (51/51), done.
remote: Compressing objects: 100% (50/50), done.
remote: Total 51 (delta 19), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (51/51), done.
Filtering content: 100% (3/3), 1.69 GiB | 76.02 MiB/s, done.


In [4]:
!wget https://public.vinai.io/PhoBERT_base_fairseq.tar.gz
!tar -xzvf PhoBERT_base_fairseq.tar.gz

--2022-07-01 14:17:59--  https://public.vinai.io/PhoBERT_base_fairseq.tar.gz
Resolving public.vinai.io (public.vinai.io)... 18.64.174.113, 18.64.174.104, 18.64.174.5, ...
Connecting to public.vinai.io (public.vinai.io)|18.64.174.113|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1243308020 (1.2G) [application/x-tar]
Saving to: ‘PhoBERT_base_fairseq.tar.gz’

PhoBERT_base_fairse 100%[===================>]   1.16G   145MB/s    in 9.1s    

2022-07-01 14:18:09 (130 MB/s) - ‘PhoBERT_base_fairseq.tar.gz’ saved [1243308020/1243308020]

PhoBERT_base_fairseq/
PhoBERT_base_fairseq/bpe.codes
PhoBERT_base_fairseq/model.pt
PhoBERT_base_fairseq/dict.txt


In [5]:
!pip install transformers==2.6.0
!pip install tf-crf-layer
!pip install fairseq
!pip install fastBPE
!pip install seqeval

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 540 kB 5.2 MB/s 
     |████████████████████████████████| 132 kB 49.9 MB/s 
     |████████████████████████████████| 880 kB 46.5 MB/s 
     |████████████████████████████████| 1.2 MB 41.0 MB/s 
     |████████████████████████████████| 5.6 MB 34.8 MB/s 
     |████████████████████████████████| 8.9 MB 38.7 MB/s 
     |████████████████████████████████| 79 kB 9.1 MB/s 
     |████████████████████████████████| 138 kB 67.7 MB/s 
     |████████████████████████████████| 127 kB 53.3 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=13b3925d0b441ddc70c27aa9a1e3a69c2bbe266a44b3cbaee81b31e1166994dd
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    U

In [6]:
!pip install fairseq
!pip install fastBPE
!pip install seqeval

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
import torch
import argparse
import numpy as np
import matplotlib.pyplot as plt
import transformers
import pandas as pd

from torch import nn
from torch.utils.data import DataLoader, RandomSampler, TensorDataset, SequentialSampler
from torch.nn import CrossEntropyLoss

from fairseq.data.encoders.fastbpe import fastBPE
from fairseq.data import Dictionary

from transformers import RobertaConfig, RobertaForTokenClassification
from transformers import AdamW
from transformers import get_linear_schedule_with_warmup

from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from tqdm.notebook import trange
from sklearn.metrics import f1_score

2022-07-01 14:19:40 | INFO | fairseq.tasks.text_to_speech | Please install tensorboardX: pip install tensorboardX


In [8]:
from transformers import *

class Ner(BertPreTrainedModel):
   config_class = RobertaConfig
   pretrained_model_archive_map = ROBERTA_PRETRAINED_MODEL_ARCHIVE_MAP
   base_model_prefix = "roberta"
   def __init__(self, config):
       super().__init__(config)
       self.num_labels = config.num_labels
       self.roberta = RobertaModel(config)
       self.dropout = nn.Dropout(config.hidden_dropout_prob)
       self.classifier = nn.Linear(config.hidden_size*4, config.num_labels)
       self.init_weights()

   def forward(
        self,
        input_ids=None,
        attention_mask=None,
        token_type_ids=None,
        position_ids=None,
        head_mask=None,
        inputs_embeds=None,
        labels=None,
    ):

       outputs = self.roberta(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
        )
       
       output = torch.cat((outputs[2][-1],outputs[2][-2], outputs[2][-3], outputs[2][-4]), dim=-1)
       sequence_output = self.dropout(output)
       logits = self.classifier(sequence_output)
       outputs = logits

       if labels is not None:
        loss_fct = CrossEntropyLoss()
        active_loss = attention_mask.view(-1) == 1
        active_logits = logits.view(-1, self.num_labels)
        active_labels = torch.where(
                active_loss, labels.view(-1), torch.tensor(loss_fct.ignore_index).type_as(labels)
        )
        loss = loss_fct(active_logits, active_labels)
        outputs = (loss, logits)
       return outputs

In [9]:
import torch

In [11]:
model2 = torch.load("/content/drive/MyDrive/Colab Notebooks/dl_project-master/ner/best_ner_syllable.pt", map_location='cpu')
model2.eval()

Ner(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(64001, 768, padding_idx=1)
      (position_embeddings): Embedding(258, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-05, elementwi

In [12]:
!git clone https://github.com/VinAIResearch/PhoNER_COVID19

Cloning into 'PhoNER_COVID19'...
remote: Enumerating objects: 44, done.
remote: Counting objects: 100% (44/44), done.
remote: Compressing objects: 100% (30/30), done.
remote: Total 44 (delta 14), reused 30 (delta 12), pack-reused 0
Unpacking objects: 100% (44/44), done.


In [13]:
df_test= []
i = 1
for line in open("/content/PhoNER_COVID19/data/word/test_word.conll", "r").readlines():
  if len(line.split())<2:
    i+=1
  else:
    tmp = line.split()
    tmp.append('s'+str(i))
    df_test.append(tmp)


In [14]:
data_test = pd.DataFrame(df_test, columns =['Word', 'Tag', 'Sentence#'])

In [15]:
def concatWord(data):
  tuple_func = lambda f: [(w, t) for w, t in zip(f['Word'].values, f['Tag'].values)]
  sentences_with_tag = data.groupby('Sentence#').apply(tuple_func)
  print(sentences_with_tag)
  sentences_with_tag = [sent for sent in sentences_with_tag]
  return sentences_with_tag

sentences_with_tag_test=concatWord(data_test)

Sentence#
s1       [(Từ, O), (24, B-DATE), (-, I-DATE), (7, I-DAT...
s10      [(Theo, O), (đó, O), (bệnh_nhân, O), (tên, O),...
s100     [(Ngoài, O), (", O), (bệnh_nhân, O), (592, B-P...
s1000    [(Theo, O), (bác_sĩ, O), (Chính, O), (,, O), (...
s1001    [(Ngày, O), (24, B-DATE), (-, I-DATE), (3, I-D...
                               ...                        
s995     [(Buổi, O), (tối, O), (đi, O), (mừng, O), (đám...
s996     [(Hiện, O), (các, O), (bệnh_nhân, O), (đang, O...
s997     [(Bệnh_nhân, O), (là, O), (mẹ, O), (và, O), (c...
s998     [(Hai, O), (bệnh_nhân, O), (được, O), (công_bố...
s999     [(Trong, O), (ngày, O), (17, B-DATE), (-, I-DA...
Length: 3000, dtype: object


In [16]:
sentences_test = [' '.join([word[0] for word in sent]) for sent in sentences_with_tag_test]
labels_test = [[word[1] for word in sent] for sent in sentences_with_tag_test]

In [17]:
sentences_test

['Từ 24 - 7 đến 31 - 7 , bệnh_nhân được mẹ là bà H.T.P ( 47 tuổi ) đón về nhà ở phường Phước_Hoà ( bằng xe_máy ) , không đi đâu chỉ ra Tạp_hoá Phượng , chợ Vườn_Lài , phường An_Sơn cùng mẹ bán tạp_hoá ở đây .',
 'Theo đó bệnh_nhân tên N.M.C. , là nhân_viên ngân_hàng tại 38 Hàng Da , phường Hàng Bông , quận Hoàn_Kiếm .',
 'Ngoài " bệnh_nhân 592 " , tại huyện Duy_Xuyên còn ghi_nhận ca nhiễm mới là " bệnh_nhân 591 " , nữ , 63 tuổi , xã Duy_Phước .',
 'Theo bác_sĩ Chính , giãn cách bệnh_nhân suy thận mạn lọc máu , nghĩa_là chia bệnh_nhân thành nhiều nhóm nhỏ ngay từ khâu cách_ly đến lọc máu ; giám_sát không cho họ tiếp_xúc gần với nhau .',
 'Ngày 24 - 3 , xét_nghiệm sàng_lọc của Trung_tâm Kiểm_soát bệnh_tật Ninh_Bình cho kết_quả dương_tính SARS - CoV - 2 .',
 '5 người về từ Thuỵ_Điển , Phần_Lan và Kuwait được Bệnh_viện Bệnh nhiệt_đới Trung_ương công_bố khỏi bệnh sáng nay .',
 '" Bệnh_nhân 812 " lây từ " bệnh_nhân 447 " ( cũng là nhân_viên tiệm bánh pizz , đi du_lịch Đà_Nẵng về ) .',
 'Theo

In [18]:
df_train = []
i = 1
for line in open("/content/PhoNER_COVID19/data/syllable/train_syllable.conll", "r").readlines():
  if len(line.split())<2:
    i+=1
  else:
    tmp = line.split()
    tmp.append('s'+str(i))
    df_train.append(tmp)
data_train = pd.DataFrame(df_train, columns =['Word', 'Tag', 'Sentence#'])
label2idx = {k: v for v, k in enumerate(data_train.Tag.unique())}
ids_to_labels = {v: k for v, k in enumerate(data_train.Tag.unique())}
label2idx['PAD']=20
label2idx['[CLS]']=21
label2idx['[SEP]']=22
label2idx['X']=23
ids_to_labels[20]='PAD'
ids_to_labels[21]='[CLS]'
ids_to_labels[22]='[SEP]'
ids_to_labels[23]='X'

In [19]:
labels_value = ['PAD', '[CLS]', '[SEP]', 'X']+data_train.Tag.unique().tolist()

In [20]:
parser = argparse.ArgumentParser()
parser.add_argument('--bpe-codes', 
    default="/content/phobert-base/bpe.codes",
    required=False,
    type=str,  
    help='path to fastBPE BPE'
)
args, unknown = parser.parse_known_args()

bpe = fastBPE(args)

In [21]:
vocab = Dictionary()
vocab.add_from_file("/content/phobert-base/vocab.txt")

def text2output(text, vocab):
  sentences_test 
  subwords_test = ['<s> '+bpe.encode(text)+' </s>']
  input_ids_test = pad_sequences([vocab.encode_line(sent, append_eos=False, add_if_not_exist=False).long().tolist() for sent in subwords_test],
                          truncating='post', padding='post', maxlen=90, value=1.0, dtype='long')
  attenion_mask_test = [[float(val != 1) for val in sent] for sent in input_ids_test]

  X_test = input_ids_test
  test_mask = attenion_mask_test

  X_test = torch.tensor(X_test)
  test_mask = torch.tensor(test_mask)
  pred_labels_ids = []
  with torch.no_grad():
    output_test = model2(X_test[0:1], test_mask[0:1])

  logit = output_test.detach().cpu().numpy()
  pred_labels_ids.extend([list(pred_label) for pred_label in np.argmax(logit, axis=2)])
  pred=[ids_to_labels[pred_labels_ids[j][i]] for j in range(len(pred_labels_ids)) for i in range(len(pred_labels_ids[j])) ]
  return list(zip(subwords_test[0].split(), pred))
  #return subwords_test[0].split(), pred





In [72]:
bang_nguyen_am = [['a', 'à', 'á', 'ả', 'ã', 'ạ', 'a'],
                  ['ă', 'ằ', 'ắ', 'ẳ', 'ẵ', 'ặ', 'aw'],
                  ['â', 'ầ', 'ấ', 'ẩ', 'ẫ', 'ậ', 'aa'],
                  ['e', 'è', 'é', 'ẻ', 'ẽ', 'ẹ', 'e'],
                  ['ê', 'ề', 'ế', 'ể', 'ễ', 'ệ', 'ee'],
                  ['i', 'ì', 'í', 'ỉ', 'ĩ', 'ị', 'i'],
                  ['o', 'ò', 'ó', 'ỏ', 'õ', 'ọ', 'o'],
                  ['ô', 'ồ', 'ố', 'ổ', 'ỗ', 'ộ', 'oo'],
                  ['ơ', 'ờ', 'ớ', 'ở', 'ỡ', 'ợ', 'ow'],
                  ['u', 'ù', 'ú', 'ủ', 'ũ', 'ụ', 'u'],
                  ['ư', 'ừ', 'ứ', 'ử', 'ữ', 'ự', 'uw'],
                  ['y', 'ỳ', 'ý', 'ỷ', 'ỹ', 'ỵ', 'y']]
bang_ky_tu_dau = ['', 'f', 's', 'r', 'x', 'j']

nguyen_am_to_ids = {}

for i in range(len(bang_nguyen_am)):
    for j in range(len(bang_nguyen_am[i]) - 1):
        nguyen_am_to_ids[bang_nguyen_am[i][j]] = (i, j)
def chuan_hoa_dau_tu_tieng_viet(word):
    if not is_valid_vietnam_word(word):
        return word
 
    chars = list(word)
    dau_cau = 0
    nguyen_am_index = []
    qu_or_gi = False
    for index, char in enumerate(chars):
        x, y = nguyen_am_to_ids.get(char, (-1, -1))
        if x == -1:
            continue
        elif x == 9:  # check qu
            if index != 0 and chars[index - 1] == 'q':
                chars[index] = 'u'
                qu_or_gi = True
        elif x == 5:  # check gi
            if index != 0 and chars[index - 1] == 'g':
                chars[index] = 'i'
                qu_or_gi = True
        if y != 0:
            dau_cau = y
            chars[index] = bang_nguyen_am[x][0]
        if not qu_or_gi or index != 1:
            nguyen_am_index.append(index)
    if len(nguyen_am_index) < 2:
        if qu_or_gi:
            if len(chars) == 2:
                x, y = nguyen_am_to_ids.get(chars[1])
                chars[1] = bang_nguyen_am[x][dau_cau]
            else:
                x, y = nguyen_am_to_ids.get(chars[2], (-1, -1))
                if x != -1:
                    chars[2] = bang_nguyen_am[x][dau_cau]
                else:
                    chars[1] = bang_nguyen_am[5][dau_cau] if chars[1] == 'i' else bang_nguyen_am[9][dau_cau]
            return ''.join(chars)
        return word
 
    for index in nguyen_am_index:
        x, y = nguyen_am_to_ids[chars[index]]
        if x == 4 or x == 8:  # ê, ơ
            chars[index] = bang_nguyen_am[x][dau_cau]
            # for index2 in nguyen_am_index:
            #     if index2 != index:
            #         x, y = nguyen_am_to_ids[chars[index]]
            #         chars[index2] = bang_nguyen_am[x][0]
            return ''.join(chars)
 
    if len(nguyen_am_index) == 2:
        if nguyen_am_index[-1] == len(chars) - 1:
            x, y = nguyen_am_to_ids[chars[nguyen_am_index[0]]]
            chars[nguyen_am_index[0]] = bang_nguyen_am[x][dau_cau]
            # x, y = nguyen_am_to_ids[chars[nguyen_am_index[1]]]
            # chars[nguyen_am_index[1]] = bang_nguyen_am[x][0]
        else:
            # x, y = nguyen_am_to_ids[chars[nguyen_am_index[0]]]
            # chars[nguyen_am_index[0]] = bang_nguyen_am[x][0]
            x, y = nguyen_am_to_ids[chars[nguyen_am_index[1]]]
            chars[nguyen_am_index[1]] = bang_nguyen_am[x][dau_cau]
    else:
        # x, y = nguyen_am_to_ids[chars[nguyen_am_index[0]]]
        # chars[nguyen_am_index[0]] = bang_nguyen_am[x][0]
        x, y = nguyen_am_to_ids[chars[nguyen_am_index[1]]]
        chars[nguyen_am_index[1]] = bang_nguyen_am[x][dau_cau]
        # x, y = nguyen_am_to_ids[chars[nguyen_am_index[2]]]
        # chars[nguyen_am_index[2]] = bang_nguyen_am[x][0]
    return ''.join(chars)
 
 
def is_valid_vietnam_word(word):
    chars = list(word)
    nguyen_am_index = -1
    for index, char in enumerate(chars):
        x, y = nguyen_am_to_ids.get(char, (-1, -1))
        if x != -1:
            if nguyen_am_index == -1:
                nguyen_am_index = index
            else:
                if index - nguyen_am_index != 1:
                    return False
                nguyen_am_index = index
    return True
 
 
def chuan_hoa_dau_cau_tieng_viet(sentence):
    """
        Chuyển câu tiếng việt về chuẩn gõ dấu kiểu cũ.
        :param sentence:
        :return:
        """
    sentence = sentence.lower()
    words = sentence.split()
    for index, word in enumerate(words):
        cw = re.sub(r'(^\p{P}*)([p{L}.]*\p{L}+)(\p{P}*$)', r'\1/\2/\3', word).split('/')
        # print(cw)
        if len(cw) == 3:
            cw[1] = chuan_hoa_dau_tu_tieng_viet(cw[1])
        words[index] = ''.join(cw)
    return ' '.join(words)
  
def chuan_hoa_chuam_cau(text):
  tmp = []
  for t in text.split():
    if t[-1] in ',.:!?;*#$%^*()!~':
      t = t[:-1]+' '+t[-1]
    tmp.append(t)
  return ' '.join(tmp)

In [114]:
text = "Ngày 13 - 3 , bệnh nhân về lại TP. HCM trên chuyến bay QH 1524 và ở tại các khách_sạn thuộc quận 1 và 4 "
text = "Bệnh nhân 24 bị đau họng tuổi ở Hà Nội là phục vụ giáo viên"
# text = "Thông báo về ca mắc mới. Tính từ 16h ngày 27/6 đến 16h ngày 28/6, trên hệ thống Quốc gia quản lý ca bệnh COVID-19 ghi nhận 769 ca nhiễm mới,"
# text = "Tình hình phòng, chống dịch bệnh covid-19 ngày 9/5"

In [115]:
import unicodedata
text = chuan_hoa_dau_tu_tieng_viet(text)
text = chuan_hoa_chuam_cau(text)
text = unicodedata.normalize('NFC', text)# NFC unicode tổ hợp sang unicode dựng sẵn

In [116]:
text2output(text, vocab)

[('<s>', '[CLS]'),
 ('Bệnh', 'O'),
 ('nhân', 'O'),
 ('24', 'B-PATIENT_ID'),
 ('bị', 'O'),
 ('đau', 'B-SYMPTOM_AND_DISEASE'),
 ('họng', 'I-SYMPTOM_AND_DISEASE'),
 ('tuổi', 'O'),
 ('ở', 'O'),
 ('Hà', 'B-LOCATION'),
 ('Nội', 'I-LOCATION'),
 ('là', 'O'),
 ('phục', 'O'),
 ('vụ', 'O'),
 ('giáo', 'I-JOB'),
 ('viên', 'O'),
 ('</s>', '[SEP]')]

In [25]:
print('tuổi'=='tuổi')

False


In [26]:
print('tuổi'.encode('utf-8'))
print('tuổi'.encode('utf-8'))

b'tu\xe1\xbb\x95i'
b'tu\xc3\xb4\xcc\x89i'


In [27]:
 import unicodedata
 a = unicodedata.normalize('NFC', 'tuổi') #NFC unicode tổ hợp sang unicode dựng sẵn
 print(a.encode('utf-8'))

b'tu\xe1\xbb\x95i'


In [28]:
text2output(a, vocab)

[('<s>', 'O'), ('tuổi', 'O'), ('</s>', 'O')]